# Correct intensities in a 3D dataset, tiled with DASK

## Dataset preparation

### Import dependencies

In [1]:
import dask.array as da
from skimage import exposure, io
from dask.distributed import Client
import numpy as np

### Activate DASK client

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 31.69 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62552,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 31.69 GiB
Comm: tcp://127.0.0.1:62575,Total threads: 4
Dashboard: http://127.0.0.1:62577/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:62555,


### Load the image and convert to a DASK array

In [3]:
# Load the image using Dask
img = da.from_array(io.imread(r"D:\fused_tp_0_ch_0.tif"))

MemoryError: Unable to allocate 236. GiB for an array with shape (126763627680,) and data type >u2

In [4]:
img

dask.array<array, shape=(865, 1384, 977), dtype=uint16, chunksize=(406, 406, 406), chunktype=numpy.ndarray>

 ### Rechunck to a desired chunk size

In [5]:
# Chunking factor

factor = np.array([10,10,10])	# even numbers

desired_chunksize = np.array(img.shape)//factor

print(desired_chunksize)

[216 346 244]


In [6]:
img_chk = img.rechunk(desired_chunksize)

In [7]:
img_chk

dask.array<rechunk-merge, shape=(865, 1384, 977), dtype=uint16, chunksize=(216, 346, 244), chunktype=numpy.ndarray>

## Set up your processing type (choose one)

### CLAHE

In [ ]:
def procedure(img_chk):
    print("proceduring", img_chk.shape)
    return exposure.equalize_adapthist(img_chk, clip_limit = 0.03)

### Histogram equalization

In [8]:
def procedure(img_chk):
    print("proceduring", img_chk.shape)
    return exposure.equalize_hist(img_chk)

### Contrast stretching

In [ ]:
def procedure(img_chk):
    print("proceduring", img_chk.shape)
    p1, p99 = np.percentile(img, (2, 98))
    return exposure.rescale_intensity(img_chk, in_range=(p1, p99))

### Adjust your overlapping

In [9]:
overlap_width = [10,10,10]
tile_map = da.map_overlap(procedure, img_chk, depth=overlap_width, boundary='nearest')

proceduring (0, 0, 0)
proceduring (1, 1, 1)


C:\Users\Usuario\anaconda3\envs\devbio-napari-env\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel, or set channel_axis.
  return func(*args, **kwargs)
C:\Users\Usuario\anaconda3\envs\devbio-napari-env\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel, or set channel_axis.
  return func(*args, **kwargs)


## Process your dataset

In [10]:
result = tile_map.compute()
print(result.dtype, result.shape, result.size)

float64 (865, 1384, 977) 1169625320


## Convert back to 16B and save your corrected dataset

In [11]:
from skimage.util import img_as_uint
img_stack_uint = img_as_uint(result)
print(img_stack_uint.dtype, img_stack_uint.shape, img_stack_uint.size)

uint16 (865, 1384, 977) 1169625320


In [12]:
io.imsave(r"E:\Users\Usuario\Desktop\221209\First batch\11-50-10_TH_WT_Blaze2_eq_2.tif", img_stack_uint)